In [1]:
from libs import *
from model import *
import json

In [2]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print("device: {}".format(device))

device: cuda:1


# Test data processing

In [3]:
matches = pd.read_csv("perf_results.csv")
M = matches['match_id'].unique()

In [4]:
len(M)

31

In [5]:
A = matches[['match_id','point_no','Alpha 1', 'Alpha 2']].loc[(matches['match_id'] == M[20])]

In [6]:
np.random.seed(0)
np.random.shuffle(M)
train_matches = M[:int(0.9*len(M))]
val_matches = M[int(0.9*len(M)):]

In [7]:
len(train_matches), len(val_matches)

(27, 4)

In [8]:
matchs = val_matches
lb = pd.read_csv("perf_results.csv")
lb = lb[lb['match_id'].isin(matchs)]

row = lb.iloc[1]
id, point_no = row['match_id'], row['point_no']
id, point_no

point_no = f'point_{point_no}'
with open(f"data/{id}.json", "r") as file:
    data = json.load(file)


In [9]:
np.array(list(data[point_no].values())).shape

(81, 2)

In [10]:
A = np.array(list(data[point_no].values())).T
result = np.zeros((400,81))
result[:A.shape[0],:A.shape[1]] = A

In [11]:
# for match_id in M:
#     df = matches.loc[(matches['match_id'] == match_id) ,:]
#     features = df.columns[4:]
    
#     dict = {}
#     for current_point in range(1, len(df) + 1):
#     # Filter data for the current point_no
#         current_data = df[df['point_no'] <= current_point]
        
#         dict[f'point_{current_point}'] = {}
        
#         for feature in features:
#             dict[f'point_{current_point}'][feature] = list(current_data[feature])
    
#     file_path = f"data/{match_id}.json"

#     with open(file_path, "w") as file:
#         json.dump(dict, file)
    
    

# Dataset

In [12]:
# Create dataset object for each point in match
class Tennis_Dataset(Dataset):
    def __init__(self, phase,  path_db, path_lb, matchs):
        super(Tennis_Dataset, self).__init__()
        self.phase = phase
        
        lb = pd.read_csv(path_lb)
        lb = lb[lb['match_id'].isin(matchs)]
        
        self.path_db = path_db
        self.result = lb
        self.output = ['Alpha 1', 'Alpha 2']
        
        self.json_data = {}
        for match_id in matchs:
            json_file_path = os.path.join(self.path_db, f"{match_id}.json")
            with open(json_file_path, "r") as file:
                self.json_data[match_id] = json.load(file)
        
    def __len__(self):
        return len(self.result)
    
    def __getitem__(self, idx):
        row = self.result.iloc[idx]
        id, point_no = row['match_id'], row['point_no']
        point_no = f'point_{point_no}'
        
        data = self.json_data[id]
        point_data = data[point_no]
        
        A = np.array(list(data[point_no].values())).T
        
        result = np.zeros((400,81))
        result[:A.shape[0],:A.shape[1]] = A
        
        labels = row[self.output].to_numpy(dtype=np.float32)
        
        return torch.from_numpy(result).float(), torch.from_numpy(labels).float()
    
    
    

In [13]:
train_dataset = Tennis_Dataset('train', 'data', "perf_results.csv", train_matches)
val_dataset = Tennis_Dataset('val', 'data', "perf_results.csv", val_matches)


In [14]:
train_dataloader = DataLoader(train_dataset, batch_size= 3, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size= 3, shuffle=False)

In [15]:
# Calculate the lengths of the train and validation datasets
train_dataset_length = len(train_dataset)
val_dataset_length = len(val_dataset)

# Calculate the percentage of validation dataset
percent_validation = (val_dataset_length / (train_dataset_length + val_dataset_length)) * 100

# Print the percentage of validation dataset
print(f"The percentage of validation dataset is: {percent_validation:.2f}%")


The percentage of validation dataset is: 13.07%


# Model for training

In [16]:
from model import Regression_sequential_data, resnet_1d

In [17]:
from utlis import Train, Test

## RNN - LSTM

In [23]:
learning_rate = 5e-4
num_hidden_units = 16
loss_function = nn.MSELoss()
epoches = 50

### RNN

In [24]:
model_RNN = Regression_sequential_data.RegressionRNN(num_sensors= 81, hidden_units=num_hidden_units).to(device)
optimizer = torch.optim.Adam(model_RNN.parameters(), lr=learning_rate)

In [25]:
import torchinfo
torchinfo.summary(model_RNN, (1, 400, 81), device= device)

Layer (type:depth-idx)                   Output Shape              Param #
RegressionRNN                            [2]                       --
├─RNN: 1-1                               [1, 400, 16]              2,128
├─Linear: 1-2                            [1, 1]                    17
Total params: 2,145
Trainable params: 2,145
Non-trainable params: 0
Total mult-adds (M): 0.85
Input size (MB): 0.13
Forward/backward pass size (MB): 0.05
Params size (MB): 0.01
Estimated Total Size (MB): 0.19

In [26]:
for epoch in range(epoches):
    Train.train_model(train_dataloader, model_RNN, epoch, loss_function, optimizer, device)
    Test.test_model(val_dataloader, model_RNN, loss_function, device)

Training epoch 0:


100%|██████████| 2111/2111 [00:50<00:00, 42.17it/s]


Train loss: 0.009518311784796386
Test loss: 0.01340980142786695
Training epoch 1:


100%|██████████| 2111/2111 [00:50<00:00, 41.72it/s]


Train loss: 0.009369410809718719
Test loss: 0.013397841048011876
Training epoch 2:


...

Train loss: 0.009202184040338426
Test loss: 0.01330084718791302
Training epoch 48:


100%|██████████| 2111/2111 [00:48<00:00, 43.22it/s]


Train loss: 0.009201108152591118
Test loss: 0.013299678129927104
Training epoch 49:


100%|██████████| 2111/2111 [00:48<00:00, 43.24it/s]


Train loss: 0.009202349612704368
Test loss: 0.013301118469607385


In [27]:
torch.save(model_RNN.state_dict(), 'result/RNN.plt')

### GRU

In [28]:
model_GRU = Regression_sequential_data.RegressionGRU(num_sensors= 81, hidden_units=num_hidden_units).to(device)
optimizer = torch.optim.Adam(model_GRU.parameters(), lr=learning_rate)

In [29]:
import torchinfo
torchinfo.summary(model_GRU, (1, 400, 81), device=device)

Layer (type:depth-idx)                   Output Shape              Param #
RegressionGRU                            [2]                       --
├─GRU: 1-1                               [1, 400, 16]              6,384
├─Linear: 1-2                            [1, 1]                    17
Total params: 6,401
Trainable params: 6,401
Non-trainable params: 0
Total mult-adds (M): 2.55
Input size (MB): 0.13
Forward/backward pass size (MB): 0.05
Params size (MB): 0.03
Estimated Total Size (MB): 0.21

In [30]:
for epoch in range(epcoches):
    Train.train_model(train_dataloader, model_GRU, epoch, loss_function, optimizer, device)
    Test.test_model(val_dataloader, model_GRU, loss_function, device)

Training epoch 0:


100%|██████████| 2111/2111 [00:49<00:00, 42.69it/s]


Train loss: 0.009413479317985163
Test loss: 0.013446425442695996
Training epoch 1:


100%|██████████| 2111/2111 [00:49<00:00, 42.83it/s]


Train loss: 0.009331849406780527
Test loss: 0.013415820699908678
Training epoch 2:


...

Train loss: 0.009199481510509045
Test loss: 0.013312171027523075
Training epoch 48:


100%|██████████| 2111/2111 [00:49<00:00, 42.79it/s]


Train loss: 0.0091933527960494
Test loss: 0.013301681555081962
Training epoch 49:


100%|██████████| 2111/2111 [00:49<00:00, 42.63it/s]


Train loss: 0.009194323262571993
Test loss: 0.013295840683935262


In [31]:
torch.save(model_GRU.state_dict(), 'result/GRU.plt')

### LSTM

In [32]:
model_LSTM = Regression_sequential_data.RegressionLSTM(num_sensors= 81, hidden_units=num_hidden_units).to(device)
optimizer = torch.optim.Adam(model_LSTM.parameters(), lr=learning_rate)

In [33]:
import torchinfo
torchinfo.summary(model_LSTM, (1, 400, 81), device=device)

Layer (type:depth-idx)                   Output Shape              Param #
RegressionLSTM                           [2]                       --
├─LSTM: 1-1                              [1, 400, 16]              8,512
├─Linear: 1-2                            [1, 1]                    17
Total params: 8,529
Trainable params: 8,529
Non-trainable params: 0
Total mult-adds (M): 3.40
Input size (MB): 0.13
Forward/backward pass size (MB): 0.05
Params size (MB): 0.03
Estimated Total Size (MB): 0.21

In [ ]:
for epoch in range(epcoches):
    Train.train_model(train_dataloader, model_LSTM, epoch, loss_function, optimizer, device)
    Test.test_model(val_dataloader, model_LSTM, loss_function, device)

In [ ]:
torch.save(model_LSTM.state_dict(), 'result/LSTM.plt')